In [1]:
import sources.logins as lg
from sqlite3 import connect
import sources.parts_list_0802 as pl
import pandas as pd
from datetime import datetime as dt
import xlwings as xw

In [2]:
conn = connect('quotation.db')

In [ ]:
q = '''
        SELECT * FROM repair_code
        WHERE [start time] IS NOT NULL
    '''
pd.read_sql(q,conn)

In [ ]:
sr = pl.summary_report()
task = sr.task_review(conn)

In [3]:
# v2
# new job no empty
q = f'''
        SELECT DISTINCT
                c.[rma no.],cu.customer_type,customer_name,cu.dealer,ac.territory,e.location,
                c.[create] as [Create Date],c.[Date Installed] AS [Installed Date],
                c.Model,c.Serial_no as Serial,sc.type,sc.mo,
                c.warranty_end_date as [Warranty End],c.recieve_date as [Received Date],
                c.in_inspect_date as [Inspection Date],c.[repair size] as [Size],c.[update user] as PIC,
                r.[start time],r.[end time],r.[start user],r.[end user],
                CASE
                    WHEN sc.status IS NULL THEN c.repair_status
                    ELSE sc.status || c.repair_status END AS [Status]

        FROM (((((consolidated c
             LEFT JOIN pf_code pf on c.[rma no.] = pf.[rma no.])
             LEFT JOIN customerS cu ON c.[customer_code] = cu.[No.])
             LEFT JOIN engineers e ON c.[update user] = e.exfm_code)
             LEFT JOIN scopes sc ON c.model = sc.model)
             LEFT JOIN acc_tbl_exp ac ON cu.[No.] = ac.[No.])
             LEFT JOIN repair_code r ON c.[rma no.] = r.[rma no.]
        WHERE c.[rma no.] > (select max(rma) from new_ml)

        ORDER BY c.[rma no.]

    '''
newjob = pd.read_sql(q,conn)
print(len(newjob))
newjob

12


,RMA No.,customer_type,CUSTOMER_NAME,dealer,Territory,location,Create Date,Installed Date,MODEL,Serial,...,Warranty End,Received Date,Inspection Date,Size,PIC,Start Time,End Time,Start User,End User,Status
0,FMSV2023100023,Government,Thai Nguyen General Hospital,TRANG THI,North,Hanoi,2023-10-16 00:00:00,2019-02-27 00:00:00,EG-250WR5,3G202K045,...,2020-02-27 00:00:00,2023-10-16 11:23:26,2023-10-18 16:46:41,Minor,nguyenminh,2023-10-18 14:47:09,2023-10-19 10:08:21,Nguyen Tuan Minh,Nguyen Tuan Minh,Shipped
1,FMSV2023100024,Government,Thai Nguyen General Hospital,TRANG THI,North,Hanoi,2023-10-16 00:00:00,2019-01-29 00:00:00,EC-250WL5,3c309k007,...,2020-01-29 00:00:00,2023-10-16 11:29:25,2023-10-18 18:56:23,Minor,nguyenminh,2023-10-18 16:56:50,2023-10-19 10:08:42,Nguyen Tuan Minh,Nguyen Tuan Minh,Shipped
2,FMSV2023100025,FF asset,FTYO - Loaner Asset,None,Loaner,HCM,2023-10-16 00:00:00,2016-12-09 00:00:00,EG-600WR,2G391K630,...,2017-12-09 00:00:00,2023-10-16 18:24:33.611000,None,None,FFAP0112,None,None,None,None,Receive
3,FMSV2023100026,None,Hai Ha District Medical Center,TRANG THI,None,Hanoi,2023-10-18 00:00:00,2019-04-25 00:00:00,EC-530FL,3C654K004,...,2020-04-25 00:00:00,2023-10-18 12:06:24.365000,None,None,tranminh,None,None,None,None,Receive
4,FMSV2023100027,Government,Vietnam – Cuba Donghoi Friendship Hospital,TRANG THI,North,Hanoi,2023-10-18 00:00:00,2016-11-11 00:00:00,EG-530WR,2G361K547,...,2017-11-11 00:00:00,2023-10-18 12:07:13,2023-10-23 16:17:40,Major,thang,None,None,None,None,Inspection
5,FMSV2023100028,None,"Nguyen Gia Pharma International Trading Co., Ltd",TRANG THI,None,Hanoi,2023-10-19 00:00:00,2020-02-06 00:00:00,EG-600WR,7G391K045,...,2021-02-06 00:00:00,2023-10-19 13:23:25.219000,None,None,thang,None,None,None,None,Receive
6,FMSV2023100029,None,Hai Hau General Hospital,TRANG THI,North,Hanoi,2023-10-19 00:00:00,2018-09-26 00:00:00,EG-530WR,4G361K130,...,2019-09-26 00:00:00,2023-10-19 13:24:48,2023-10-23 13:39:20,Major,thang,None,None,None,None,Inspection
7,FMSV2023100030,Private,Viet Nhat Clinic 3 - Dr.Giang'S Clinic,TRANG THI,North,Hanoi,2023-10-19 00:00:00,2014-10-25 00:00:00,EG-530UR2,RU039A048,...,2015-10-25 00:00:00,2023-10-19 13:30:58,None,None,thang,None,None,None,None,Receive
8,FMSV2023100031,Government,National Lung Hospital,TRANG THI,North,Hanoi,2023-10-23 00:00:00,2016-12-29 00:00:00,EB-530T,1B084K400,...,2017-12-29 00:00:00,2023-10-23 17:09:32,None,None,thang,None,None,None,None,Receive
9,FMSV2023100032,Government,Viet Duc Hospital,TRANG THI,North,Hanoi,2023-10-23 00:00:00,2023-02-03 00:00:00,EC-760R-V/I,8C728K200,...,None,2023-10-23 17:32:50.502000,None,None,tranminh,None,None,None,None,Receive


In [ ]:
# add missed rma by sn
sn_checked = tuple(['3C309K012',''
                   ])
q = f'''
        SELECT DISTINCT
                c.[rma no.],cu.customer_type,customer_name,cu.dealer,ac.territory,e.location,
                c.[create] as [Create Date],c.[Date Installed] AS [Installed Date],
                c.Model,c.Serial_no as Serial,sc.type,sc.mo,
                c.warranty_end_date as [Warranty End],c.recieve_date as [Received Date],
                c.in_inspect_date as [Inspection Date],c.[repair size] as [Size],c.[update user] as PIC,
                CASE
                    WHEN sc.status IS NULL THEN c.repair_status
                    ELSE sc.status || c.repair_status END AS [Status]
        
        FROM ((((consolidated c
             LEFT JOIN pf_code pf on c.[rma no.] = pf.[rma no.])
             LEFT JOIN customerS cu ON c.[customer_code] = cu.[No.])
             LEFT JOIN engineers e ON c.[update user] = e.exfm_code)
             LEFT JOIN scopes sc ON c.model = sc.model)
             LEFT JOIN acc_tbl_exp ac ON cu.[No.] = ac.[No.]
        WHERE c.serial_no IN {sn_checked}
        ORDER BY c.[rma no.]
            
    '''
newjob = pd.read_sql(q,conn)
print(len(newjob))
newjob

In [ ]:
pd.read_sql('SELECT [status] FROM new_ml',conn)

In [4]:
# add new job into ML

m_list = pd.read_sql('SELECT * FROM new_ml',conn)
bb = pl.summary_report()
cur = conn.cursor()
for i in range(len(newjob)):
    rma = newjob.iloc[i]['RMA No.']
    issue = bb.issues(conn,rma)
    i_no = i + len(m_list) + 1
    for j in range(len(newjob.iloc[i])):
        if newjob.iloc[i][j] == None:newjob.iloc[i][j] =''

    cur.execute(f"""INSERT INTO new_ml(rma,cus_type,Customer,Dealer,Location,installed_date,
                                       model,sn,scope_type,model2,wty_end_date,receive,inspection,repair_size,issue,
                                       pic,[status]) 
                    VALUES('{newjob.iloc[i]['RMA No.']}','{newjob.iloc[i]['customer_type']}',"{newjob.iloc[i]['CUSTOMER_NAME']}",
                    '{newjob.iloc[i]['dealer']}','{newjob.iloc[i]['location']}',
                    '{str(newjob.iloc[i]['Installed Date'])}','{newjob.iloc[i]['MODEL']}','{newjob.iloc[i]['Serial']}',
                    '{newjob.iloc[i]['Type']}','{newjob.iloc[i]['mo']}','{newjob.iloc[i]['Warranty End']}',
                    '{newjob.iloc[i]['Received Date']}','{str(newjob.iloc[i]['Inspection Date'])}','{newjob.iloc[i]['Size']}',
                    "{issue}","{newjob.iloc[i]['PIC']}","{newjob.iloc[i]['Status']}")
    """)

# save conn
conn.commit()
print('Done')

Done


In [5]:
# update job v2
q = '''select max([UPDATE_time]) AS max_update FROM new_ml'''
udt = pd.read_sql(q,conn)
update_time = udt['max_update'][0]
print(update_time)

# filter new update in consolidated 
cur = conn.cursor()
q = f'''

        SELECT c.[rma no.],c.customer_name,c.model,c.serial_no,

                c.recieve_date,c.[repair size],c.[update user],c.in_inspect_date,c.repair_status,c.[status info],
                c.[other id],c.last_update_time,c.approval,c.[repair user name],
                c.[update time],r.[start time],r.[end time],r.[start user],r.[end user]
        FROM consolidated c
        LEFT JOIN repair_code r ON c.[rma no.] = r.[rma no.]
        WHERE c.[update time] > '{update_time}'
        AND C.[rma no.] IN (SELECT rma FROM new_ml)

        ORDER BY 1

    '''
update_job = pd.read_sql(q,conn)
print(len(update_job))
update_job

2023-10-16 10:37:38.361000
56


,RMA No.,CUSTOMER_NAME,MODEL,SERIAL_NO,RECIEVE_DATE,Repair Size,Update User,IN_INSPECT_DATE,REPAIR_STATUS,Status Info,Other ID,LAST_UPDATE_TIME,Approval,Repair User Name,Update Time,Start Time,End Time,Start User,End User
0,FMSV2023020022,FTYO - Loaner Asset,EC-760ZP-V/M,2C730K032,2023-02-08 18:54:02,Major,thang,2023-02-14 12:09:53,Shipped,PO Received,None,2023-10-23 19:53:02.989000,Cancel,None,2023-10-23 19:53:02.989000,None,None,None,None
1,FMSV2023020022,FTYO - Loaner Asset,EC-760ZP-V/M,2C730K032,2023-02-08 18:54:02,Major,thang,2023-02-14 12:09:53,Shipped,PO Received,None,2023-10-23 19:53:02.989000,Cancel,None,2023-10-23 19:53:02.989000,None,None,None,None
2,FMSV2023020022,FTYO - Loaner Asset,EC-760ZP-V/M,2C730K032,2023-02-08 18:54:02,Major,thang,2023-02-14 12:09:53,Shipped,PO Received,None,2023-10-23 19:53:02.989000,Cancel,None,2023-10-23 19:53:02.989000,None,None,None,None
3,FMSV2023040004,FTYO - Loaner Asset,EN-580T,1C675K021,2023-04-03 17:13:24,Major,thang,2023-04-11 13:22:41,Shipped,PO Received,None,2023-10-23 19:55:31.555000,Cancel,None,2023-10-23 19:55:31.555000,None,None,None,None
4,FMSV2023040004,FTYO - Loaner Asset,EN-580T,1C675K021,2023-04-03 17:13:24,Major,thang,2023-04-11 13:22:41,Shipped,PO Received,None,2023-10-23 19:55:31.555000,Cancel,None,2023-10-23 19:55:31.555000,None,None,None,None
5,FMSV2023070039,Hanoi Medical University Hospital,VP-4450HD,2V567K687,2023-07-14 17:06:42,Other,thang,2023-07-21 19:12:48,Authorization,Parts on Order,WTY-FFVN-2307024R,2023-10-16 17:07:28.831000,Approval,None,2023-10-16 17:07:28.831000,2023-10-16 15:07:01,None,Nguyen Khac Thang (Hanoi),None
6,FMSV2023080003,Phu Tho General Hospital,EG-530WR,JG361K553,2023-08-01 18:59:40,Major,FFAP0070,2023-08-08 12:42:38,Repair,None,WTY-FFVN-2308029,2023-10-18 11:12:05.240000,Approval,Nguyen Tuan Minh,2023-10-18 11:12:05.240000,2023-10-16 08:20:38,2023-10-18 09:12:00,Nguyen Tuan Minh,Nguyen Tuan Minh
7,FMSV2023080003,Phu Tho General Hospital,EG-530WR,JG361K553,2023-08-01 18:59:40,Major,FFAP0070,2023-08-08 12:42:38,Repair,None,WTY-FFVN-2308029,2023-10-18 11:12:05.240000,Approval,Nguyen Tuan Minh,2023-10-18 11:12:05.240000,2023-10-16 08:20:38,2023-10-18 09:12:00,Nguyen Tuan Minh,Nguyen Tuan Minh
8,FMSV2023080003,Phu Tho General Hospital,EG-530WR,JG361K553,2023-08-01 18:59:40,Major,FFAP0070,2023-08-08 12:42:38,Repair,None,WTY-FFVN-2308029,2023-10-18 11:12:05.240000,Approval,Nguyen Tuan Minh,2023-10-18 11:12:05.240000,2023-10-16 08:20:38,2023-10-18 09:12:00,Nguyen Tuan Minh,Nguyen Tuan Minh
9,FMSV2023080003,Phu Tho General Hospital,EG-530WR,JG361K553,2023-08-01 18:59:40,Major,FFAP0070,2023-08-08 12:42:38,Repair,None,WTY-FFVN-2308029,2023-10-18 11:12:05.240000,Approval,Nguyen Tuan Minh,2023-10-18 11:12:05.240000,2023-10-16 08:20:38,2023-10-18 09:12:00,Nguyen Tuan Minh,Nguyen Tuan Minh


In [6]:
# update all
bb = pl.summary_report() # create issue
cur = conn.cursor()
m_list = pd.read_sql('SELECT * FROM new_ml',conn)
for i in range(len(update_job)):
# i = 0

    rma = update_job.iloc[i]['RMA No.']
    issue = bb.issues(conn,rma)
    print(issue)
    #replace none to empty
    for j in range(len(update_job.iloc[i])):
        if update_job.iloc[i][j] == None:update_job.iloc[i][j] =''

    # UPDATE GENERAL
    q = f""" UPDATE new_ml 
                SET receive = '{update_job.iloc[i]["RECIEVE_DATE"]}',
                    inspection = '{update_job.iloc[i]['IN_INSPECT_DATE']}',
                    exfm_status = '{update_job.iloc[i]['REPAIR_STATUS']}',
                    approval = '{update_job.iloc[i]['Approval']}',
                    repair_user = '{update_job.iloc[i]['Repair User Name']}',
                    update_time = '{update_job.iloc[i]['Update Time']}',
                    update_user = '{update_job.iloc[i]['Update User']}',
                    exfm_info = '{update_job.iloc[i]['Status Info']}',
                    start_repair = '{update_job.iloc[i]['Start Time']}',
                    start_repair2 = '{update_job.iloc[i]['Start Time']}',
                    end_repair = '{update_job.iloc[i]['End Time']}'
                WHERE RMA = '{rma}'"""
    cur.execute(q)
    #update fixed item
    fixed_items = ['receive','report_id','inspection','repair_size']
    update_items = ['RECIEVE_DATE','Other ID','IN_INSPECT_DATE','Repair Size']
#     update_items[fixed_items.index('Report_ID')]       # compare each point

    for item in fixed_items:
        if str(m_list[m_list['rma']==rma][item])=='':
            cur.execute(f'''
                            UPDATE new_ml SET '{item}' = '{updatejob.iloc[i][update_items[fixed_items.index(item)]]}'
                            WHERE RMA = '{rma}'
                        ''')
            print(f'Update {rma}: {item} values {updatejob.iloc[i][update_items[fixed_items.index(item)]]}')
conn.commit()

LGB, BSA, FSB
LGB, BSA, FSB
LGB, BSA, FSB
FCT, BSA
FCT, BSA
PCB
FCT, CHA, FSA, VCA
FCT, CHA, FSA, VCA
FCT, CHA, FSA, VCA
FCT, CHA, FSA, VCA
BSA
DWA
FSB, DWA
FSB, DWA
CHA
BSA, DEC, FCT, LGB, FSB
BSA, DEC, FCT, LGB, FSB
BSA, DEC, FCT, LGB, FSB
BSA, DEC, FCT, LGB, FSB
BSA, DEC, FCT, LGB, FSB
BSA, DWA, FSA
BSA, DWA, FSA
BSA, DWA, FSA
IRS
FSB, VCA, PCB, NOZ
FSB, VCA, PCB, NOZ
FSB, VCA, PCB, NOZ
FSB, VCA, PCB, NOZ
IRS
CHA, FCT
CHA, FCT
AWT, CHA, LGB, BSA
AWT, CHA, LGB, BSA
AWT, CHA, LGB, BSA
AWT, CHA, LGB, BSA
ANGL
VCA, FSA, WGA
VCA, FSA, WGA
VCA, FSA, WGA
CHA, OSA
CHA, OSA
OSA
OSA


FSB

AWT, CHA, FCT, BSA
AWT, CHA, FCT, BSA
AWT, CHA, FCT, BSA
AWT, CHA, FCT, BSA







In [9]:
len(stt_info)

1030

In [10]:
#v2
# update empty status
bb = pl.summary_report() # create issue
cur = conn.cursor()
m_list = pd.read_sql('SELECT * FROM new_ml',conn)

q = f'''
        SELECT DISTINCT
        m.rma,m.customer,m.model,m.sn,C.[repair_status],c.[status info],m.exfm_status,m.exfm_info,
        c.approval,c.[update time],c.[update user],c.[repair user name],c.recieve_date,c.in_inspect_date,
        c.[repair size],r.[start time],r.[end time],r.[start user],r.[end user]

        FROM (new_ml m
        LEFT JOIN consolidated c ON m.rma = c.[rma no.])
        LEFT JOIN repair_code r ON r.[rma no.] = c.[rma no.]

    '''
stt_info = pd.read_sql(q,conn)
display(stt_info)

for i in range(len(stt_info)):
# i = 0
#     print(i)
    rma = m_list.iloc[i]['rma']
    issue = bb.issues(conn,rma)
#     print(rma,issue)

    #replace none to empty
    for j in range(len(stt_info.iloc[i])):
        if stt_info.iloc[i][j] == None:stt_info.iloc[i][j] =''

    # UPDATE GENERAL
    cur.execute(f"""UPDATE new_ml SET issue = '{issue}',
    exfm_status = '{stt_info.iloc[i]['REPAIR_STATUS']}',
    exfm_info = '{stt_info.iloc[i]['Status Info']}',
    approval = '{stt_info.iloc[i]['Approval']}',
    update_time = '{stt_info.iloc[i]['Update Time']}',
    update_user = '{stt_info.iloc[i]['Update User']}',
    receive = '{stt_info.iloc[i]['RECIEVE_DATE']}',
    inspection = '{stt_info.iloc[i]['IN_INSPECT_DATE']}',
    repair_size = '{stt_info.iloc[i]['Repair Size']}',
    start_repair2 = '{stt_info.iloc[i]['Start Time']}',
    end_repair = '{stt_info.iloc[i]['End Time']}'

    WHERE rma = '{rma}'""")

conn.commit()
print('Done')

,rma,customer,model,sn,REPAIR_STATUS,Status Info,exfm_status,exfm_info,Approval,Update Time,Update User,Repair User Name,RECIEVE_DATE,IN_INSPECT_DATE,Repair Size,Start Time,End Time,Start User,End User
0,FMSV2023020022,FTYO - Loaner Asset,EC-760ZP-V/M,2C730K032,Shipped,PO Received,Shipped,PO Received,Cancel,2023-10-23 19:53:02.989000,thang,None,2023-02-08 18:54:02,2023-02-14 12:09:53,Major,None,None,None,None
1,FMSV2023030060,Hoang Long Clinic,VP-7000,2V644K028,Inspection,Parts on Order,Shipped,PO Received,None,2023-08-09 11:49:51.353000,thang,None,2023-03-23 19:41:03,2023-03-31 12:26:02,Other,None,None,None,None
2,FMSV2023040004,FTYO - Loaner Asset,EN-580T,1C675K021,Shipped,PO Received,Shipped,PO Received,Cancel,2023-10-23 19:55:31.555000,thang,None,2023-04-03 17:13:24,2023-04-11 13:22:41,Major,None,None,None,None
3,FMSV2023050025,Hoang Long Clinic,EG-760R,1G402K303,Inspection,Parts on Order,Inspection,Parts on Order,None,2023-08-09 11:50:32.098000,thang,None,2023-05-10 12:39:50,2023-05-17 12:42:16,Major,None,None,None,None
4,FMSV2023050078,Thai Binh Hospital,EG-530WR,LG361K357,Inspection,Parts on Order,Shipped,PO Received,None,2023-10-01 22:40:04.133000,thongle,None,2023-05-30 20:46:15,2023-06-01 19:14:12,Minor,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,FMSV2023100030,Viet Nhat Clinic 3 - Dr.Giang'S Clinic,EG-530UR2,RU039A048,Receive,None,Parts Selection,Awating Contract billing,None,2023-10-23 18:56:33.092000,thang,None,2023-10-19 13:30:58,None,None,None,None,None,None
320,FMSV2023100031,National Lung Hospital,EB-530T,1B084K400,Receive,None,Parts Selection,Awating Contract billing,None,2023-10-23 17:09:50.803000,thang,None,2023-10-23 17:09:32,None,None,None,None,None,None
321,FMSV2023100032,Viet Duc Hospital,EC-760R-V/I,8C728K200,Receive,None,Parts Selection,Awating Contract billing,None,2023-10-23 17:32:50.502000,tranminh,None,2023-10-23 17:32:50.502000,None,None,None,None,None,None
322,FMSV2023100033,Viet Duc Hospital,EC-760R-V/I,8C728K201,Receive,None,Parts Selection,Awating Contract billing,None,2023-10-23 17:33:28.324000,tranminh,None,2023-10-23 17:33:28.324000,None,None,None,None,None,None


Done


In [11]:
#check no receive and inspection
q = '''
        SELECT rma,receive,inspection,
        c.[recieve_date],c.[in_inspect_date],m.issue,m.repair_size,c.[repair size]
        
        FROM new_ml m
        LEFT JOIN consolidated c ON m.rma = c.[rma no.]
        WHERE (receive IS NULL OR inspection IS NULL)
        
    '''
receive_inspect = pd.read_sql(q,conn)
receive_inspect

,rma,receive,inspection,RECIEVE_DATE,IN_INSPECT_DATE,issue,repair_size,Repair Size


In [ ]:
# add receive inspect infomation
# receive_inspect
for i in range(len(receive_inspect)):
# i = 0
#     print(i)
    rma = receive_inspect.iloc[i]['rma']
    issue = bb.issues(conn,rma)
#     print(rma,issue)
    
    #replace none to empty
    for j in range(len(receive_inspect.iloc[i])):
        if receive_inspect.iloc[i][j] == None:receive_inspect.iloc[i][j] =''

    # UPDATE GENERAL
    cur.execute(f"""UPDATE new_ml SET issue = '{issue}',
    
    receive = '{receive_inspect.iloc[i]['RECIEVE_DATE']}',
    inspection = '{receive_inspect.iloc[i]['IN_INSPECT_DATE']}',
    repair_size = '{receive_inspect.iloc[i]['Repair Size']}'
    
    WHERE rma = '{rma}'""")
    
conn.commit()
print('Done')

In [ ]:
#filter completed
q = '''
        SELECT  rma,customer,model, sn,repair_size,issue,pic,note,location,
            receive,quotation,confirmation,part_order_date, part_received_date,
            start_repair,qc_date,return_date 
        FROM new_ml 
        WHERE status LIKE '%Completed repair' 
        ORDER BY rma
'''
completed_list = pd.read_sql(q,conn)
completed_list

In [ ]:
#delete job completed from new_ml
cur.execute('DELETE FROM new_ml WHERE status LIKE "%Completed repair"')
conn.commit()

In [ ]:
q = '''
        SELECT * FROM new_ml
        WHERE exfm_status IS NULL
        
    '''
pd.read_sql(q,conn)

In [ ]:
# 
q = "SELECT * FROM new_ml where exfm_status is null"
pd.read_sql(q,conn)

#####  Export


In [12]:
file_name = lg.file_select(folder_name = 'files', end_with = '.xlsm')
xw.Book(file_name)


______________________________________________________________________
|  No.|  File Name                               |  Modified Time    |
|____________________________________________________________________|
|  1  |  New ML_231011_110731.xlsm               |  23-Oct-23 11:57  |
|  2  |  New ML_231017_130751.xlsm               |  23-Oct-23 17:53  |
|____________________________________________________________________|
Select file (Default 1): 2
"New ML_231017_130751.xlsm" is selected


<Book [New ML_231017_130751.xlsm]>

In [13]:
# final
q = '''SELECT * FROM new_ml'''
m_list_final = pd.read_sql(q,conn)
for i in range(len(m_list_final)):
    rma = m_list_final.iloc[i]['rma']
    issue = bb.issues(conn,rma)

    #replace none to empty
    for j in range(len(m_list_final.iloc[i])):
#         if m_list_final.iloc[i][j] == None: m_list_final.iloc[i][j] =''
        if m_list_final.iloc[i][j] == 'None': m_list_final.iloc[i][j] =''

m_list_final = m_list_final.drop(['TAT_receive','TAT_PO','TAT_PART'],axis=1)
dtmp = dt.now().strftime('%y%m%d')
file_name = f'ML_{dtmp}.xlsx'
m_list_final.to_excel(file_name,index = False)
xw.Book(file_name)

<Book [ML_231023.xlsx]>

In [ ]:
# copy file from files folder to new ML
file = lg.file_select(folder_name='files',start_with='New ML')
# to destination folder

try:
    copy_selected_file(des_folder='New ML',file_name = file)
    print(f'Copy sucess file {file}')
except Exception as e:
    print(e)


In [ ]:
# delete Master list in folder file
import os
import pathlib
path = pathlib.Path().absolute()
folder_name = 'files'
file_name = file.split('\\')[-1]
for file in os.listdir(folder_name):
    
    if 'New ML' in file and file_name not in file:
        print(f'deleted file {file}')
        os.remove(os.path.join(path,folder_name,file))
        
# delete file ML
for file in os.listdir():
    
    if 'ML_' in file and file_name not in file:
        print(f'deleted file {file}')
        os.remove(os.path.join(path,file))

for file in os.listdir('New ML'):
    
    if 'ML_' in file and file_name not in file:
        print(f'deleted file {file}')
        os.remove(os.path.join(path,file))

In [ ]:
for file in os.listdir('New ML'):
    
    if 'ML_' in file and file_name not in file:
        print(f'deleted file {file}')
        os.remove(os.path.join(path,file))

In [ ]:
# compare new ML and ExFM
q = '''
        SELECT [rma no.],customer_name, model, serial_no,repair_status,
        [update user],[update time],[status info]
        
        FROM consolidated
        WHERE [rma no.] NOT IN (SELECT rma FROM new_ml)
        
'''
gap = pd.read_sql(q,conn)
gap.to_excel('exfm_gap_230816.xlsx',index = False)

In [ ]:
q = f'''
        SELECT distinct rma,customer,nm.dealer,cs.dealer,c.customer_code
        
        FROM (new_ml nm
        LEFT JOIN consolidated c ON nm.rma = c.[rma no.])
        LEFT JOIN customers cs ON c.customer_code = cs.[no.]
    '''
d =pd.read_sql(q,conn)
d.to_excel('inspection/dealer.xlsx',index = False)

In [ ]:
q = 'SELECT DEALER from customers WHERE [NO.] = "FMSV00100"'
pd.read_sql(q,conn)

In [ ]:
conn.close()
conn = connect('quotation.db')

In [ ]:
conn.close()